In [ ]:
# 📦 1. 필요한 패키지 설치 (처음 한 번만 실행)
# %pip install httpx folium python-dotenv nest_asyncio

In [ ]:
# 🔐 2. 환경 변수 또는 직접 키 입력
import os
import pandas as pd
import folium
import math
import json
import random
import colorsys
from dotenv import load_dotenv

load_dotenv()
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")

In [ ]:
# 출발지/도착지 좌표 설정
origin = {"x": 126.93287655929, "y": 37.484396394971}      # 예: 양지병원
destination = {"x": 126.92417006127, "y": 37.493027115674}  # 예: 보라매병원
# origin = {"x": 127.031886816408, "y": 37.471958811697}      # 예: 경부고속도로
# destination = {"x": 127.037463917199, "y": 37.4720040276214}  # 예: 양재역

priority = "RECOMMEND"

url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}",
    "Content-Type": "application/json"
}
data = {
    "origin": origin,
    "destination": destination,
    "priority": priority
}

async def fetch_route():
    async with httpx.AsyncClient() as client:
        params = {
            "origin": f"{origin['x']},{origin['y']}",
            "destination": f"{destination['x']},{destination['y']}",
            "priority": priority
        }

        response = await client.get(
            url, headers=headers, params=params
        )
        response.raise_for_status()
        return response.json()

route_data = await fetch_route()

roads = route_data["routes"][0]["sections"][0]["roads"]
guides = route_data["routes"][0]["sections"][0]["guides"]

# 파일 로드
protection_path = "../data/external/protection_zone_data.csv"
crosswalk_path = "../data/external/crosswalk_data.csv"
traffic_light_path = "../data/external/traffic_light_data.csv"

protection_df = pd.read_csv(protection_path)
crosswalk_df = pd.read_csv(crosswalk_path)
traffic_light_df = pd.read_csv(traffic_light_path)

In [ ]:
### 필요함수정의 ###
def haversine(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: m)
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)

    a = math.sin(d_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # 거리 (m)

# 거리 계산 (빠른 하버사인)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(d_lambda/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def interpolate_points(lat1, lon1, lat2, lon2, interval=5):
    distance = haversine(lat1, lon1, lat2, lon2)
    steps = max(1, int(distance // interval))
    lat_points = [lat1 + (lat2 - lat1) * i / steps for i in range(1, steps)]
    lon_points = [lon1 + (lon2 - lon1) * i / steps for i in range(1, steps)]
    return list(zip(lat_points, lon_points))

def is_within_haversine(center, coord_list, radius_m):
    return [coord for coord in coord_list if haversine(center[0], center[1], coord[0], coord[1]) <= radius_m]

In [ ]:
coords = []
for road in roads:
    vertexes = road["vertexes"]
    coords += [(vertexes[i+1], vertexes[i]) for i in range(0, len(vertexes), 2)]

mid = coords[len(coords)//2]
fmap = folium.Map(location=mid, zoom_start=15)

folium.PolyLine(coords, color="blue", weight=5).add_to(fmap)

folium.Marker(coords[0], tooltip="출발지", icon=folium.Icon(color="green")).add_to(fmap)
folium.Marker(coords[-1], tooltip="도착지", icon=folium.Icon(color="red")).add_to(fmap)

fmap

In [ ]:
def random_color():
    h = random.random()              # 색상(Hue)
    s = 0.9                          # 채도 높게
    v = 0.9                          # 명도 높게
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return '#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255))

for i, road in enumerate(roads):
    vertexes = road["vertexes"]
    coords = [(vertexes[i+1], vertexes[i]) for i in range(0, len(vertexes), 2)]

    folium.PolyLine(
        coords,
        color=random_color(),
        weight=5,
        tooltip=f"{road.get('name', '구간')} / {road.get('distance', 0)}m"
    ).add_to(fmap)

folium.Marker(coords[0], tooltip="출발지", icon=folium.Icon(color="green")).add_to(fmap)
folium.Marker(coords[-1], tooltip="도착지", icon=folium.Icon(color="red")).add_to(fmap)

fmap

In [171]:
points = []
for road in roads:
    vertexes = road["vertexes"]
    coords = [(vertexes[i+1], vertexes[i]) for i in range(0, len(vertexes), 2)]
    points.extend(coords)

mid = points[len(points)//2]
fmap = folium.Map(location=mid, zoom_start=15)

point_idx = 1  # 전체 좌표 번호
for road in roads:
    vertexes = road["vertexes"]
    coords = [(vertexes[i+1], vertexes[i]) for i in range(0, len(vertexes), 2)]

    color = random_color()

    for lat, lon in coords:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            tooltip=f"Point {point_idx}"
        ).add_to(fmap)
        point_idx += 1
   
# guide 점 찍기     
for g in guides:
    lat = g["y"]
    lon = g["x"]
    name = g.get("name", "지점명 없음")
    guidance = g.get("guidance", "안내 없음")
    dist = g.get("distance", 0)
    duration = g.get("duration", 0)
    type = g.get("type", 0)
    type_message = ""
    
    if type == 2:
        type_message = "우회전 포함"
    if type == 3:
        type_message = "U턴 포함" 
    if 30<=type<=41:
        type_message = "로터리 포함"
    if 70<=type<=81:
        type_message = "회전 교차로 포함"


    tooltip_text = (
        f"<b>{name}</b><br>"
        f"{guidance}<br>"
        f"거리: {dist}m<br>"
        f"기타: {type_message}!"
    )

    folium.CircleMarker(
        location=(lat, lon),
        radius=6,
        color="purple",
        fill=True,
        fill_color="purple",
        fill_opacity=0.8,
        tooltip=folium.Tooltip(tooltip_text, sticky=True)
    ).add_to(fmap)
        
fmap


In [164]:
# folium 초기화
fmap = folium.Map(location=mid, zoom_start=15)

# 점 찍기
for idx, (lat, lon) in enumerate(points):
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8
    ).add_to(fmap)

# 점과 점 사이 원 그리기
for i in range(len(points) - 1):
    lat1, lon1 = points[i]
    lat2, lon2 = points[i + 1]

    # 중심점 (중간 위치)
    center_lat = (lat1 + lat2) / 2
    center_lon = (lon1 + lon2) / 2

    # 거리 계산 (미터 단위)
    radius = haversine(lat1, lon1, lat2, lon2) / 2

    # folium 원 그리기
    folium.Circle(
        location=(center_lat, center_lon),
        radius=radius,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.3
    ).add_to(fmap)

fmap


In [167]:
# points = []
# for road in roads:
#     v = road["vertexes"]
#     points += [(v[i+1], v[i]) for i in range(0, len(v), 2)]

# 보간 점 추가
points = []
for road in roads:
    v = road["vertexes"]
    for i in range(0, len(v) - 2, 2):
        lat1, lon1 = v[i+1], v[i]
        lat2, lon2 = v[i+3], v[i+2]
        points.append((lat1, lon1))  # 시작점
        mids = interpolate_points(lat1, lon1, lat2, lon2, interval=25)
        points.extend(mids)          # 중간점
    # 마지막 점은 따로 추가
    points.append((v[-1], v[-2]))


# 중심점 계산
mid = points[len(points)//2]

# ✅ 시각화 여부 토글
SHOW_PROTECTION_MARKER = False
SHOW_CROSSWALK_MARKER = False
SHOW_TRAFFIC_LIGHT_MARKER = False

# folium 지도 초기화
fmap = folium.Map(location=mid, zoom_start=17)

# 1. 점-점 사이 원 그리고 색상 표시
for i in range(len(points) - 1):
    lat1, lon1 = points[i]
    lat2, lon2 = points[i + 1]
    
    center = ((lat1 + lat2) / 2, (lon1 + lon2) / 2)
    radius = haversine(lat1, lon1, lat2, lon2) / 2

    in_protection = is_within_haversine(center, protection_coords, radius)
    in_crosswalk = is_within_haversine(center, crosswalk_coords, radius)
    in_traffic = is_within_haversine(center, traffic_light_coords, radius)

    # 존재 여부 확인
    exists = [bool(in_protection), bool(in_crosswalk), bool(in_traffic)]
    count = sum(exists)

    # 색상 지정
    if count == 3:
        color = "red"
    elif count == 2:
        color = "orange"
    elif count == 1:
        color = "yellow"
    else:
        color = "green"

    # 🟢 원 그리기
    folium.Circle(
        location=center,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3
    ).add_to(fmap)

    # 🧩 조건부 마커 표시
    if SHOW_PROTECTION_MARKER:
        for lat, lon in in_protection:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="green", icon="plus", prefix="fa"),
                tooltip=f"보호구역\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

    if SHOW_CROSSWALK_MARKER:
        for lat, lon in in_crosswalk:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="blue", icon="road", prefix="fa"),
                tooltip=f"횡단보도\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

    if SHOW_TRAFFIC_LIGHT_MARKER:
        for lat, lon in in_traffic:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="red", icon="lightbulb", prefix="fa"),
                tooltip=f"신호등\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

# 2. 모든 좌표 점 찍기
for lat, lon in points:
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8
    ).add_to(fmap)

# 지도 시각화
fmap

In [169]:
points = []
for road in roads:
    v = road["vertexes"]
    points += [(v[i+1], v[i]) for i in range(0, len(v), 2)]

# 중심점 계산
mid = points[len(points)//2]

# ✅ 시각화 여부 토글
SHOW_PROTECTION_MARKER = False
SHOW_CROSSWALK_MARKER = False
SHOW_TRAFFIC_LIGHT_MARKER = False

# folium 지도 초기화
fmap1 = folium.Map(location=mid, zoom_start=17)

# 1. 점-점 사이 원 그리고 색상 표시
for i in range(len(points) - 1):
    lat1, lon1 = points[i]
    lat2, lon2 = points[i + 1]
    
    center = ((lat1 + lat2) / 2, (lon1 + lon2) / 2)
    radius = haversine(lat1, lon1, lat2, lon2) / 2

    in_protection = is_within_haversine(center, protection_coords, radius)
    in_crosswalk = is_within_haversine(center, crosswalk_coords, radius)
    in_traffic = is_within_haversine(center, traffic_light_coords, radius)

    # 존재 여부 확인
    exists = [bool(in_protection), bool(in_crosswalk), bool(in_traffic)]
    count = sum(exists)

    # 색상 지정
    if count == 3:
        color = "red"
    elif count == 2:
        color = "orange"
    elif count == 1:
        color = "yellow"
    else:
        color = "green"

    # 🟢 원 그리기
    folium.Circle(
        location=center,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3
    ).add_to(fmap1)

    # 🧩 조건부 마커 표시
    if SHOW_PROTECTION_MARKER:
        for lat, lon in in_protection:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="green", icon="plus", prefix="fa"),
                tooltip=f"보호구역\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap1)

    if SHOW_CROSSWALK_MARKER:
        for lat, lon in in_crosswalk:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="blue", icon="road", prefix="fa"),
                tooltip=f"횡단보도\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap1)

    if SHOW_TRAFFIC_LIGHT_MARKER:
        for lat, lon in in_traffic:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="red", icon="lightbulb", prefix="fa"),
                tooltip=f"신호등\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap1)

# 2. 모든 좌표 점 찍기
for lat, lon in points:
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8
    ).add_to(fmap1)

# 지도 시각화
fmap1

In [170]:
from IPython.display import HTML, display

html1 = fmap._repr_html_()
html2 = fmap1._repr_html_()

display(HTML(f"""
<div style="display: flex; justify-content: space-between; gap: 10px;">
    <div style="width: 49%;">{html1}</div>
    <div style="width: 49%;">{html2}</div>
</div>
"""))


In [ ]:
points = []
for road in roads:
    v = road["vertexes"]
    for i in range(0, len(v) - 2, 2):
        lat1, lon1 = v[i+1], v[i]
        lat2, lon2 = v[i+3], v[i+2]
        points.append((lat1, lon1))  # 시작점
        mids = interpolate_points(lat1, lon1, lat2, lon2, interval=25)
        points.extend(mids)          # 중간점
    # 마지막 점은 따로 추가
    points.append((v[-1], v[-2]))


# 중심점 계산
mid = points[len(points)//2]

# ✅ 시각화 여부 토글
SHOW_PROTECTION_MARKER = False
SHOW_CROSSWALK_MARKER = False
SHOW_TRAFFIC_LIGHT_MARKER = False

# folium 지도 초기화
fmap = folium.Map(location=mid, zoom_start=17)

# 1. 점-점 사이 원 그리고 색상 표시
for i in range(len(points) - 1):
    lat1, lon1 = points[i]
    lat2, lon2 = points[i + 1]
    
    center = ((lat1 + lat2) / 2, (lon1 + lon2) / 2)
    radius = haversine(lat1, lon1, lat2, lon2) / 2

    in_protection = is_within_haversine(center, protection_coords, radius)
    in_crosswalk = is_within_haversine(center, crosswalk_coords, radius)
    in_traffic = is_within_haversine(center, traffic_light_coords, radius)

    # 존재 여부 확인
    exists = [bool(in_protection), bool(in_crosswalk), bool(in_traffic)]
    count = sum(exists)

    # 색상 지정
    if count == 3:
        color = "red"
    elif count == 2:
        color = "orange"
    elif count == 1:
        color = "yellow"
    else:
        color = "green"

    # 🟢 원 그리기
    folium.Circle(
        location=center,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3
    ).add_to(fmap)

    # 🧩 조건부 마커 표시
    if SHOW_PROTECTION_MARKER:
        for lat, lon in in_protection:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="green", icon="plus", prefix="fa"),
                tooltip=f"보호구역\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

    if SHOW_CROSSWALK_MARKER:
        for lat, lon in in_crosswalk:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="blue", icon="road", prefix="fa"),
                tooltip=f"횡단보도\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

    if SHOW_TRAFFIC_LIGHT_MARKER:
        for lat, lon in in_traffic:
            folium.Marker(
                location=(lat, lon),
                radius=5, 
                icon=folium.Icon(color="red", icon="lightbulb", prefix="fa"),
                tooltip=f"신호등\n({lat:.6f}, {lon:.6f})"
            ).add_to(fmap)

# 2. 모든 좌표 점 찍기
for lat, lon in points:
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8
    ).add_to(fmap)

# 지도 시각화
fmap